## Experimented with POS tags and gensim to perform aspect-level sentiment analysis on restaurant reviews but decided to change the approach.

In [13]:
import csv
import nltk
import gensim
from nltk.tokenize import word_tokenize
from nltk.tokenize import StanfordTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords


In [22]:
fname = r"reviews_ChinesePN_1000.csv"
f = open(fname, 'r', encoding = 'utf8')
reader = csv.reader(f)
#separate yelp reviews and target sentiment value
def separateTextTarget(reader):
    docs = []
    target = []    
    for row in reader:
        break
    for row in reader:
        docs.append(row[0])
        target.append(row[1])
    return (docs, target)
docs, target = separateTextTarget(reader)
f.close()

In [23]:
#define tokenizer
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer = StanfordTokenizer('../stanford-postagger/stanford-postagger.jar')
tokenized_text = tokenizer.tokenize(docs[0]) #test

In [41]:
#tokenize text-- by sentence and by words within the sentence
#listSentListWords: list of lists-- lists of tokenized words
listSentListWords = []
words = []
for review in docs:
    sentTok = sent_detector.tokenize(review)
    temp = []
    for i in sentTok:
        w = word_tokenize(i)
        words.extend(w)
        temp.append(w)
    listSentListWords.extend(temp)
print(len(listSentListWords))

8083


In [42]:
model = gensim.models.Word2Vec(iter=1, min_count=2)  # an empty model, no training yet
model.build_vocab(listSentListWords)

In [43]:
#remove stopwords
stop_words = set(stopwords.words('english'))
words = [w.lower() for w in words if not w.lower() in stop_words and len(w) > 1]
#find most common words -- to select nouns (restaurant aspects)
# and to find close words using word vectors/synonyms/similarity scores
common = nltk.FreqDist(words).most_common(1000)
topwords = [w[0] for w in common]
tagged = nltk.pos_tag(topwords)
print(tagged[:5])

[('food', 'NN'), ('good', 'JJ'), ("n't", 'RB'), ('place', 'NN'), ("'s", 'POS')]


In [44]:
#find most commonly occuring nouns
nouns = [w for w,tag in tagged if tag in ['NN', 'NNP', 'NNS']]
nouns_freq = [(w,freq) for w,freq in common if w in nouns]
print(nouns_freq[:5])

[('food', 1156), ('place', 637), ('chicken', 477), ('service', 367), ('rice', 353)]


In [45]:
#separate data into test/training
n = int(len(listSentListWords)*0.8)
trainSent = listSentListWords[:n]
testSent = listSentListWords[n:]

In [46]:
#test out tools in gensim
model = gensim.models.Word2Vec(iter=1)  # an empty model, no training yet
model.build_vocab(trainSent)  # can be a non-repeatable, 1-pass generator
model.train(testSent)  # can be a non-repeatable, 1-pass generator

14599

In [47]:
model = gensim.models.Word2Vec(iter=1, min_count=1)  # an empty model, no training yet
model.build_vocab(listSentListWords)

In [48]:
x = gensim.models.Word2Vec(listSentListWords[:5000])
x.train(listSentListWords[5000:])

138185

In [30]:
model.train(vocabset1)

12507

In [49]:
x = gensim.models.Word2Vec(listSentListWords[:5000])
x.train(listSentListWords[5000:])

137826